In [1]:
import pandas as pd
import sqlite3
import os

# =========================================================
# STEP 0: Remove old database (prevents table already exists error)
# =========================================================
if os.path.exists("restaurants.db"):
    os.remove("restaurants.db")

# =========================================================
# STEP 1: Load orders.csv
# =========================================================
orders = pd.read_csv("orders.csv")
print("Orders Data:")
print(orders.head())

# =========================================================
# STEP 2: Load users.json
# =========================================================
users = pd.read_json("users.json")
print("\nUsers Data:")
print(users.head())

# =========================================================
# STEP 3: Load restaurants.sql into SQLite
# =========================================================
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

# =========================================================
# STEP 4: Read restaurants table
# =========================================================
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
print("\nRestaurants Data:")
print(restaurants.head())

# =========================================================
# STEP 5: Merge datasets (LEFT JOIN)
# =========================================================
merged_df = orders.merge(users, on="user_id", how="left")
final_df = merged_df.merge(restaurants, on="restaurant_id", how="left")

# =========================================================
# STEP 6: Save final dataset
# =========================================================
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("\n✅ Final dataset created successfully!")
print("Rows:", final_df.shape[0])
print("Columns:", final_df.shape[1])

conn.close()

# =========================================================
# ======================= ANALYSIS ========================
# =========================================================

# 1️⃣ Gold member revenue by city
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print("\nGold Member Revenue by City:")
print(gold_city_revenue)

# 2️⃣ Average order value by cuisine
avg_cuisine_value = (
    final_df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print("\nAverage Order Value by Cuisine:")
print(avg_cuisine_value)

# 3️⃣ Number of users spending more than ₹1000
user_spending = final_df.groupby("user_id")["total_amount"].sum()
high_spenders = user_spending[user_spending > 1000]

print("\nNumber of users spending > 1000:")
print(high_spenders.count())

# 4️⃣ Gold members – average order value by city
gold_city_avg = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

print("\nGold Members - Average Order Value by City:")
print(gold_city_avg)

# 5️⃣ Revenue by restaurant rating range
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

rating_revenue = (
    final_df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print("\nRevenue by Rating Range:")
print(rating_revenue)

# 6️⃣ Cuisine with few restaurants but strong revenue
cuisine_stats = final_df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
)

print("\nCuisine Stats:")
print(cuisine_stats.sort_values("distinct_restaurants"))

# 7️⃣ Percentage of orders by Gold members
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])
percentage = round((gold_orders / total_orders) * 100)

print("\nPercentage of orders by Gold members:")
print(percentage, "%")

# 8️⃣ Restaurant with highest avg order value (< 20 orders)
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    order_count=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)

filtered_restaurants = restaurant_stats[
    restaurant_stats["order_count"] < 20
].sort_values("avg_order_value", ascending=False)

print("\nHighest Avg Order Value Restaurants (<20 orders):")
print(filtered_restaurants.head(10))

# 9️⃣ Revenue by Membership + Cuisine
combo_revenue = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print("\nRevenue by Membership + Cuisine:")
print(combo_revenue)

# 🔟 Revenue by Quarter
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

quarter_revenue = (
    final_df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

print("\nRevenue by Quarter:")
print(quarter_revenue)

Orders Data:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  

Users Data:
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold

Restaurants Data:
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1 

C:\Users\navee\AppData\Local\Temp\ipykernel_23456\1496756453.py:116: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"]
